# TeslaMax Maps

In [1]:
from pathlib import Path
import os

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize, differential_evolution, basinhopping
from pandas import Series, DataFrame
import pandas as pd

import teslamax
from teslamax import TeslaMaxGeometry, TeslaMaxPreDesign, TeslaMaxModel


In [2]:
os.chdir(str(Path.home() / "code" / "TeslaMax"))

In [3]:
os.getcwd()

'M:\\code\\TeslaMax'

## Maps for R_s

In [4]:
DICT_LINES = 19

In [5]:
filename = "map_Rs_Ro_50.txt"

In [6]:
file_path = Path(filename)

In [7]:
with open(file_path) as f:
    flines = f.readlines()

In [8]:
params = eval("".join(flines[0:DICT_LINES]))

In [9]:
params

{'B_rem_II_1': 1.4,
 'B_rem_II_2': 1.4,
 'B_rem_IV_1': 1.4,
 'B_rem_IV_2': 1.4,
 'B_rem_IV_3': 1.4,
 'R_e': 0.3,
 'R_i': 0.015,
 'R_o': 0.05,
 'h_fc': 0.01,
 'h_gap': 0.025,
 'linear_iron': 1,
 'mu_r_II': 1.05,
 'mu_r_IV': 1.05,
 'mu_r_iron': 5000.0,
 'n_II': 2,
 'n_IV': 3,
 'phi_C_II': 15,
 'phi_S_II': 45,
 'phi_S_IV': 45}

In [10]:
n = params["n_II"] + params["n_IV"]

In [11]:
["alpha_II_%d" %(i+1) for i in range(0,params["n_II"])]

['alpha_II_1', 'alpha_II_2']

In [12]:
column_names = ["B_max[T]",
               "F_M[%]",
               "R_s[mm]",
               "K[]"] + (
    ["alpha_II_%d" %(i+1) for i in range(0,params["n_II"])] + 
    ["alpha_IV_%d" %(j+1) for j in range(0,params["n_IV"])]) 

In [13]:
column_names

['B_max[T]',
 'F_M[%]',
 'R_s[mm]',
 'K[]',
 'alpha_II_1',
 'alpha_II_2',
 'alpha_IV_1',
 'alpha_IV_2',
 'alpha_IV_3']

In [14]:
map_table = pd.read_csv(filename,sep='\t',skiprows=20,names=column_names,header=0)
map_table

,B_max[T],F_M[%],R_s[mm],K[],alpha_II_1,alpha_II_2,alpha_IV_1,alpha_IV_2,alpha_IV_3
0,1.200,35.0,120.0,0.440,0.000,0.000,8.213,32.736,88.701
1,1.200,37.5,120.0,0.521,0.000,0.000,7.982,31.364,88.251
2,1.200,40.0,120.0,0.648,0.000,0.000,7.795,30.266,87.567
3,1.200,42.5,120.0,0.824,0.000,0.000,7.649,29.423,86.802
4,1.200,45.0,130.0,0.630,18.424,1.819,8.892,30.876,87.844
5,1.225,35.0,120.0,0.559,0.000,0.000,8.431,33.251,88.958
6,1.225,37.5,120.0,0.648,0.000,0.000,8.200,31.966,88.522
7,1.225,40.0,120.0,0.783,0.000,0.000,8.010,30.933,87.869
8,1.225,42.5,120.0,0.966,0.000,0.000,7.860,30.131,87.148
9,1.225,45.0,130.0,0.718,17.478,0.000,8.832,31.486,88.448


In [15]:
B_max = map_table["B_max[T]"]

In [16]:
B_max

0     1.200
1     1.200
2     1.200
3     1.200
4     1.200
5     1.225
6     1.225
7     1.225
8     1.225
9     1.225
10    1.250
11    1.250
12    1.250
13    1.250
14    1.250
15    1.275
16    1.275
17    1.275
18    1.275
19    1.275
20    1.300
21    1.300
22    1.300
23    1.300
24    1.300
Name: B_max[T], dtype: float64

In [17]:
F_M = map_table["F_M[%]"]

In [18]:
R_s = map_table["R_s[mm]"]